In [1]:
from fractal.triangle_folding import unfold_iso_triangle, triangle_grid
from fractal.geometry_fractal import reg_poly_fractal
from fractal.com_fractal import com_fractal
from fractal.appollonian import chaos_map_iter
from fractal.animation import animate
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

In [2]:
# Increase chunksize to avoid memory issues
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 1000000

In [3]:
# warpage is the % of a circle the triangle is folded into
# 1.0 means the triangle is folded fully into a circular, 0.5 means semi-circles, etc
warpage = 1.0

In [4]:
# Different functions for radial folding
sine1 = lambda t: 0.5*(np.sin(np.pi*t)+1)
triwave1 = lambda t: 0.5*(signal.sawtooth(np.pi*t, 0.5)+1)
sine2 = lambda t: 0.5*np.sin(np.pi*t)
triwave2 = lambda t: 0.5*signal.sawtooth(np.pi*t, 0.5)
tan1 = lambda t: 0.5*(np.tan(np.pi*t)+1)

custom_map = triwave1

# Sierpinski Triangle Folding Animation

In [5]:
# generate a Sierpinski triangle
sierpinski = reg_poly_fractal(3, 13, scaling=0.5, rotation=0)

# determine its maximum height (the max radius of the circle when its folded)
max_r = sierpinski[:,1].max()

# Unfold the Sierpinski triangle into a circle and then apply custom_maps (which progressively fold the radius more over time)
# This generates a list of coordinate points to be plotted
disc_shapes = [unfold_iso_triangle(sierpinski, warpage=warpage, custom_map=lambda r,theta: [max_r*custom_map(a*r/max_r), theta])
                for a in np.linspace(1, 1.75, num=200)]

In [6]:
# Animate by sequentially plotting each coordinate set in the list over time, and saving to a gif
plot_kwargs={"ls":'', "marker":'.', "markersize":0.01, "linewidth": 0.1, "alpha":0.2}
animate(disc_shapes, "sierpinski_triwave1.gif", fps=60, windowsize=[[-3*np.pi, 3*np.pi], [-3*np.pi, 3*np.pi]], figsize=[6,6], keep_axis=False, plot_kwargs=plot_kwargs)

# Grid in Triangle Folding Animation

In [7]:
# Generate a gridded triangle (X_hor for horizontal gridlines, X_vert for vertical gridlines)
X_hor, X_vert = triangle_grid(n_gridlines=21, n_points=100001)

# Determine max height of gridlines (which will become the max height of the folded circle)
max_r = max(X_vert[:,1].max(), X_hor[:,1].max())

# Unfold the gridded triangle's horizontal and vertical lines into a circle 
# and apply custom_maps (which progressively fold the radius more over time)
# Note that disc_shapes is now a list of lists of form [X_hor, X_vert] where each is an array of 2d coordinates
disc_shapes = [[unfold_iso_triangle(X_hor, warpage=warpage, custom_map=lambda r,theta: [max_r*custom_map(a*r/max_r), theta]),
                unfold_iso_triangle(X_vert, warpage=warpage, custom_map=lambda r,theta: [max_r*custom_map(a*r/max_r), theta])]
                for a in np.linspace(1, 5, num=200)]

In [8]:
# Animate by sequentially plotting each coordinate set in the list over time, and saving to a gif
plot_kwargs={"ls":'', "marker":'.', "markersize":1, "alpha":0.1}
animate(disc_shapes, "triangle_grid_triwave1.gif", fps=60, windowsize=None, figsize=[8,8], keep_axis=False, plot_kwargs=plot_kwargs)

# Appollonian Gasket Folding Animation

In [9]:
# Generate Appollonian Gasket via Chaos Game
appollonian = chaos_map_iter(iters=500000, n_inputs_reused=5)

100%|██████████| 500000/500000 [00:54<00:00, 9110.64it/s]


In [10]:
# Helper functions to determine the radius,
# (the Appollonian gasket automatically starts as a circle so we need to determine its polar coords to do radial folding)
def from_2d(coords):
    # Converts (?, 2) array to two (?,) arrays
    return coords[:,0], coords[:,1]

def to_2d(x, y):
    # Converts two (?,) arrays to a (?, 2) array
    return np.concatenate([x.reshape(-1,1), y.reshape(-1,1)], axis=1)

def cartesian_to_polar(coords):
    x, y = from_2d(coords)
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    return to_2d(r, theta)

def polar_to_cartesian(coords):
    r, theta = from_2d(coords)
    x = r*np.cos(theta)
    y = r*np.sin(theta)
    return to_2d(x, y)

In [11]:
# Determine polar coordinates and radius
apolarnian = cartesian_to_polar(appollonian)
max_r = appollonian[:,0].max()

# Define function to alter the radius of the polar coords and convert to Cartesian
def radius_rotate(coords_polar, a):
    new_r = max_r*custom_map(a*coords_polar[:,0]/max_r).astype("float32")
    new_polar = to_2d(new_r, coords_polar[:,1])
    return polar_to_cartesian(new_polar)

# Alter the polar coords ot the Appollonian gasket (which progressively fold the radius more over time)
appollonian_shapes = [radius_rotate(apolarnian, a) for a in np.linspace(1, 5, num=200)]

In [12]:
# Animate by sequentially plotting each coordinate set in the list over time, and saving to a gif
plot_kwargs = {"ls":'', "marker":'.', "markersize":0.025}
animate(appollonian_shapes, "appollonian_triwave1.gif", fps=60, figsize=[6,6], keep_axis=False, plot_kwargs=plot_kwargs)